In [1]:
import wandb

In [2]:
# choose labels subset
ekman_emotions = False
FSJ_emotions = True
FSJN_emotions = False
all_emotions = False
full_ekman_emotions = False
sentiment = False
sentiment_3 = False

In [3]:
all_emotions_list = ['admiration', 'amusement',
       'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
       'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
       'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
       'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
       'sadness', 'surprise', 'neutral']

In [4]:
import pickle5 as pkl
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

/home/svetlana.maslenkova/.conda/envs/nlp_project_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os, sys

cwd = os.getcwd()
cwd

'/home/svetlana.maslenkova/Documents/nlp_project'

In [6]:
EMO_DATA_PATH = cwd + '/data/go_emotions_dataset.csv'

In [7]:
row_data = pd.read_csv(EMO_DATA_PATH)
# remove unclear samples
row_data = row_data[row_data.example_very_unclear==False]
# remove duplicated samples
row_data = row_data.drop_duplicates(subset='text', keep='first')

print(row_data.shape)
print(row_data.columns)
row_data

(57730, 31)
Index(['id', 'text', 'example_very_unclear', 'admiration', 'amusement',
       'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
       'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
       'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
       'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
       'sadness', 'surprise', 'neutral'],
      dtype='object')


,id,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,eew5j0j,That game hurt.,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,ed2mah1,"You do right, if you don't care then fuck 'em!",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,eeibobj,Man I love reddit.,False,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,eda6yn6,"[NAME] was nowhere near them, he was by the Fa...",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,eespn2i,Right? Considering it’s such an important docu...,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211088,ef0lnq5,I didn’t even notice that one. I noticed the b...,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
211094,edirq0m,"Thanks, [NAME]",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
211095,ee6pagw,Everyone likes [NAME].,False,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
211138,eegg1aj,Projecting pretty hard here.,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
emo_count = row_data[['admiration', 'amusement',
       'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
       'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
       'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
       'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
       'sadness', 'surprise', 'neutral']].sum().sort_values(ascending=False)

emo_count

neutral           15803
approval           4898
admiration         4874
annoyance          3534
gratitude          3468
disapproval        3106
curiosity          2808
amusement          2684
optimism           2448
realization        2385
love               2370
anger              2242
joy                2205
disappointment     2193
confusion          2023
sadness            1851
caring             1611
excitement         1558
surprise           1528
disgust            1368
desire             1014
fear                891
remorse             714
embarrassment       637
nervousness         489
relief              372
pride               354
grief               180
dtype: int64

In [11]:
if sentiment:
    neg_emo = [ 'annoyance', 'disappointment', 'sadness', 'disapproval', 'anger', 'sadness', 'disgust', 'fear', 'embarrassment', 'grief']
    neu_emo = ['neutral']
    pos_emo = ['approval', 'admiration', 'gratitude', 'curiosity', 'optimism', 'love', 'joy', 'caring', 'excitement', 'surprise', 'pride', 'relief', ]
    mix_emo = ['amusement', 'realization', 'confusion',  'desire', 'remorse', 'nervousness']
    
if full_ekman_emotions:
    anger_emo = ["anger", "annoyance", "disapproval"]
    disgust_emo = ["disgust"]
    fear_emo = ["fear", "nervousness"]
    joy_emo =  ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire", "caring"]
    sadness_emo = ["sadness", "disappointment", "embarrassment", "grief",  "remorse"]
    surprise_emo = ["surprise", "realization", "confusion", "curiosity"]
    neutral_emo = ['neutral']

if sentiment_3:
    positive = ["amusement", "excitement", "joy", "love", "desire", "optimism", "caring", "pride", "admiration", "gratitude", "relief", "approval"]
    negative = ["fear", "nervousness", "remorse", "embarrassment", "disappointment", "sadness", "grief", "disgust", "anger", "annoyance", "disapproval"]
    ambiguous = ["realization", "surprise", "curiosity", "confusion"]

In [12]:
# emo_df = row_data[['text'] + all_emotions_list].copy()
if sentiment_3:
    emo_df = row_data[['text'] + positive +negative+ambiguous].copy()

if ekman_emotions:
    emo_df = row_data[['text']+['joy',  'fear', 'anger',  'surprise', 'disgust',  'sadness']].copy()

if FSJ_emotions:
    emo_df = row_data[['text']+['fear', 'sadness', 'joy']].copy()
    emo_df = emo_df[emo_df[['fear', 'sadness', 'joy']].sum(axis=1)>0]

if FSJN_emotions:
    emo_df = row_data[['text']+['fear', 'sadness', 'joy', 'neutral']].copy()
    emo_df = emo_df[emo_df[['fear', 'sadness', 'joy', 'neutral']].sum(axis=1)>0]

In [13]:
if sentiment:
    S = row_data[neg_emo].copy()
    neg_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[neu_emo].copy()
    neu_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[pos_emo].copy()
    pos_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[mix_emo].copy()
    mix_ids = S[S.sum(axis=1)>0].index.to_list()

if full_ekman_emotions:
    S = row_data[anger_emo].copy()
    anger_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[disgust_emo].copy()
    disguist_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[fear_emo].copy()
    fear_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[joy_emo].copy()
    joy_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[sadness_emo].copy()
    sadness_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[surprise_emo].copy()
    surprise_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[neutral_emo].copy()
    neutral_ids = S[S.sum(axis=1)>0].index.to_list()

if sentiment:
    S = row_data[positive].copy()
    positive_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[negative].copy()
    negative_ids = S[S.sum(axis=1)>0].index.to_list()
    S = row_data[ambiguous].copy()
    ambiguous_ids = S[S.sum(axis=1)>0].index.to_list()

if FSJN_emotions:
    neu_ids = emo_df[emo_df.neutral==1].sample(frac=0.1).index.to_list()
    fear_ids = emo_df[emo_df.fear==1].index.to_list()
    sadness_ids = emo_df[emo_df.sadness==1].index.to_list()
    joy_ids = emo_df[emo_df.joy==1].index.to_list()
    emo_df = emo_df[emo_df.index.isin(neu_ids+fear_ids+sadness_ids+joy_ids)].copy().sample(frac=1)

In [14]:
if four_cat_emotions:
    pos = [1 if id in pos_ids else 0 for id in emo_df.index]
    neg = [1 if id in neg_ids else 0 for id in emo_df.index]
    neu = [1 if id in neu_ids else 0 for id in emo_df.index]
    mix = [1 if id in mix_ids else 0 for id in emo_df.index]

if full_ekman_emotions:
    anger = [1 if id in anger_ids else 0 for id in emo_df.index]
    disguist = [1 if id in disguist_ids else 0 for id in emo_df.index]
    fear = [1 if id in fear_ids else 0 for id in emo_df.index]
    joy = [1 if id in joy_ids else 0 for id in emo_df.index]
    sadness = [1 if id in sadness_ids else 0 for id in emo_df.index]
    surprise = [1 if id in surprise_ids else 0 for id in emo_df.index]
    neutral = [1 if id in neutral_ids else 0 for id in emo_df.index]

if sentiment:
    pos = [1 if id in positive_ids else 0 for id in emo_df.index]
    neg = [1 if id in negative_ids else 0 for id in emo_df.index]
    ambig = [1 if id in ambiguous_ids else 0 for id in emo_df.index]

In [15]:
if sentiment:
    emo_df = row_data[['text']].copy()
    emo_df['positive'] = pos
    emo_df['negative'] = neg
    emo_df['neutral'] = neu
    emo_df['mixed'] = mix

if full_ekman_emotions:
    emo_df = row_data[['text']].copy()
    emo_df['anger'] = anger
    emo_df['disguist'] = disguist
    emo_df['joy'] = joy
    emo_df['sadness'] = sadness
    emo_df['surprise'] = surprise
    emo_df['neutral'] = neutral

if sentiment_3:
    emo_df = row_data[['text']].copy()
    emo_df['positive'] = pos
    emo_df['negative'] = neg
    emo_df['ambiguous'] = ambig

emo_df.head()

,text,fear,sadness,joy
0,That game hurt.,0,1,0
29,So happy for [NAME]. So sad he's not here. Ima...,0,1,1
43,By far the coolest thing I've seen on this thr...,0,0,1
49,Sending love and strength vibes <3,0,0,1
58,Me too! First time in a couple of years!,0,0,1


In [29]:
if ekman_emotions:
    emo_df = row_data[['text', 'joy', 'fear', 'anger', 'surprise', 'disgust', 'sadness']].copy()
    emo_df = emo_df[emo_df[['joy', 'fear', 'anger', 'surprise', 'disgust', 'sadness']].sum(axis=1)>0]

if all_emotions:    
    emo_df = row_data[['text', 'admiration', 'amusement',
        'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
        'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
        'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love',
        'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse',
        'sadness', 'surprise', 'neutral']].copy()
       
print(emo_df.shape)

(35400, 7)


In [30]:
emo_df['WORD_COUNT'] = emo_df['text'].apply(lambda x: len(x.split()))

In [15]:
if FSJ_emotions:
    emo_df['target_list'] = list(zip(emo_df.fear, emo_df.sadness, emo_df.joy))
if FSJN_emotions:
    emo_df['target_list'] = list(zip(emo_df.fear, emo_df.sadness, emo_df.joy, emo_df.neutral))
if ekman_emotions:
    emo_df['target_list'] = list(zip(emo_df.joy, emo_df.fear, emo_df.anger, emo_df.surprise, emo_df.disgust, emo_df.sadness))
if sentiment:
    emo_df['target_list'] = list(zip(emo_df.positive, emo_df.negative, emo_df.neutral, emo_df.mixed))
if all_emotions:    
    emo_df['target_list'] = list(zip(emo_df.admiration, emo_df.amusement, emo_df.anger, emo_df.annoyance, \
                            emo_df.approval, emo_df.caring, emo_df.confusion, emo_df.curiosity, \
                                emo_df.desire, emo_df.disappointment, emo_df.disapproval, emo_df.disgust,\
                                     emo_df.embarrassment, emo_df.excitement, emo_df.fear, emo_df.gratitude, \
                                         emo_df.grief, emo_df.joy, emo_df.love, emo_df.nervousness, emo_df.optimism, \
                                             emo_df.pride, emo_df.realization, emo_df.relief, emo_df.remorse,\
                                                 emo_df.sadness, emo_df.surprise, emo_df.neutral))

if full_ekman_emotions:
    emo_df['target_list'] = list(zip(emo_df.anger, emo_df.disguist, emo_df.joy, emo_df.sadness, emo_df.surprise, emo_df.neutral))

if sentiment_3:
    emo_df['target_list'] = list(zip(emo_df.positive, emo_df.negative, emo_df.ambiguous))


In [32]:
emo_df.head(2)

,text,fear,sadness,joy,target_list
0,That game hurt.,0,1,0,"(0, 1, 0)"
29,So happy for [NAME]. So sad he's not here. Ima...,0,1,1,"(0, 1, 1)"


In [33]:
emo_df.shape

(35400, 9)

In [17]:
# calculate weights for the loss

pos_weights = []

if FSJ_emotions:
    for emotion in ['fear',	'sadness',	'joy']:
        pos_weights.append(np.round(np.sum(emo_df[emotion]==0) / np.sum(emo_df[emotion]==1),2))   

if FSJN_emotions:
    for emotion in ['fear',	'sadness',	'joy', 'neutral']:
        pos_weights.append(np.round(np.sum(emo_df[emotion]==0) / np.sum(emo_df[emotion]==1),2))   
        
if ekman_emotions:
    pos_weights.append(np.round(np.sum(emo_df.joy==0) / np.sum(emo_df.joy==1),2))
    pos_weights.append(np.round(np.sum(emo_df.fear==0) / np.sum(emo_df.fear==1)))
    pos_weights.append(np.round(np.sum(emo_df.anger==0) / np.sum(emo_df.anger==1)))
    pos_weights.append(np.round(np.sum(emo_df.surprise==0) / np.sum(emo_df.surprise==1)))
    pos_weights.append(np.round(np.sum(emo_df.disgust==0) / np.sum(emo_df.disgust==1)))
    pos_weights.append(np.round(np.sum(emo_df.sadness==0) / np.sum(emo_df.sadness==1)))

if sentiment:
    pos_weights.append(np.round(np.sum(emo_df.positive==0) / np.sum(emo_df.positive==1),2))
    pos_weights.append(np.round(np.sum(emo_df.negative==0) / np.sum(emo_df.negative==1),2))
    pos_weights.append(np.round(np.sum(emo_df.neutral==0) / np.sum(emo_df.neutral==1),2))
    pos_weights.append(np.round(np.sum(emo_df.mixed==0) / np.sum(emo_df.mixed==1),2))

if all_emotions:
    for emotion in all_emotions_list:
        pos_weights.append(np.round(np.sum(emo_df[emotion]==0) / np.sum(emo_df[emotion]==1),2))
    
if full_ekman_emotions:
    for emotion in ['anger',	'disguist',	'joy',	'sadness',	'surprise',	'neutral']:
        pos_weights.append(np.round(np.sum(emo_df[emotion]==0) / np.sum(emo_df[emotion]==1),2))

if sentiment_3:
    for emotion in ['positive',	'negative',	'ambiguous']:
        pos_weights.append(np.round(np.sum(emo_df[emotion]==0) / np.sum(emo_df[emotion]==1),2))   


pos_weights

[4.53, 1.61, 1.21]

In [18]:
from transformers import BertTokenizer

# Defining some key variables that will be used later on in the training
MAX_LEN = 16
BATCH_SIZE = 900
EPOCHS = 30
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['text']
        self.targets = self.data.target_list
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index]).lower()
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [20]:
from sklearn.model_selection import train_test_split

train_dataset, temp_dataset = train_test_split(emo_df, random_state=43, test_size = 0.1)
temp_dataset = temp_dataset.reset_index(drop=True)
valid_dataset, test_dataset = train_test_split(temp_dataset, random_state=43, test_size = 0.5)
valid_dataset = valid_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(emo_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALID Dataset: {}".format(valid_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
validation_set = CustomDataset(valid_dataset, tokenizer, MAX_LEN)
test_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

train_loader = DataLoader(training_set, batch_size=BATCH_SIZE , shuffle=True)
valid_loader = DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

FULL Dataset: (17669, 5)
TRAIN Dataset: (15902, 5)
VALID Dataset: (883, 5)
TEST Dataset: (884, 5)


In [22]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=128,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)
        self.l3 = torch.nn.Linear(256, 3)
    
    def forward(self, ids, mask, token_type_ids):
        _, output= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output = self.l2(output).unsqueeze(1)
        output, _ = self.lstm(output)
        output = self.l3(output).squeeze(1)
        return output

model = BERTClass()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Functions

In [23]:
def load_ckp(checkpoint_fpath, model, optimizer, gpu=True):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    if gpu==False:
      map_location=torch.device('cpu')
    else:
      map_location = torch.device('cuda')
    # load check point
    checkpoint = torch.load(checkpoint_fpath, map_location=map_location)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min#.item()

In [24]:
import shutil, sys   
def save_ckp(state, is_best, checkpoint_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    # if is_best:
    #     best_fpath = best_model_path
    #     # copy that checkpoint file to best path given, best_model_path
    #     shutil.copyfile(f_path, best_fpath)

In [25]:
def train_model(start_epochs,  n_epochs, valid_loss_min_input, 
                training_loader, validation_loader, model, 
                optimizer, checkpoint_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = valid_loss_min_input 
   
 
  for epoch in range(start_epochs, n_epochs+1):
    running_train_loss = 0.0
    running_valid_loss = 0
    global_step = 0
    epoch_patience = 0
    valid_targets = []

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(torch.sigmoid(outputs), targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        running_train_loss += loss.item() * targets.size(0)
        global_step += 1
        #print('after loss data in training', loss.item(), train_loss)
        wandb.log({'train_loss':loss.item(), 'global_step':global_step})
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(torch.sigmoid(outputs), targets)
            running_valid_loss += loss.item() * targets.size(0)
            # valid_targets.extend(targets.cpu().detach().numpy().tolist())
            # val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = running_train_loss/len(training_loader.dataset)
      valid_loss = running_valid_loss/len(validation_loader.dataset)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      wandb.log({'epoch_train_loss':train_loss, 'epoch_val_loss':valid_loss, 'epoch':epoch})
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      # save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path)
        valid_loss_min = valid_loss
        epoch_patience = 0
      else:
        epoch_patience += 1

    if epoch_patience >= 3:
      break

    print('############# Epoch {}  Done   #############\n'.format(epoch))


  return model

### Training

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [27]:
model = BERTClass()
model = model.to(device)

In [29]:
# print model parametes
params_to_update = model.parameters()
counter = 0

for name, param in model.named_parameters():
    if counter > (4): # here we specify layers to unfreeze
        param.requires_grad = False
        print('\t', name, param.requires_grad)

    else:
        param.requires_grad = True
        print('\t', name, param.requires_grad)
    counter += 1


	 l1.embeddings.word_embeddings.weight True
	 l1.embeddings.position_embeddings.weight True
	 l1.embeddings.token_type_embeddings.weight True
	 l1.embeddings.LayerNorm.weight True
	 l1.embeddings.LayerNorm.bias True
	 l1.encoder.layer.0.attention.self.query.weight False
	 l1.encoder.layer.0.attention.self.query.bias False
	 l1.encoder.layer.0.attention.self.key.weight False
	 l1.encoder.layer.0.attention.self.key.bias False
	 l1.encoder.layer.0.attention.self.value.weight False
	 l1.encoder.layer.0.attention.self.value.bias False
	 l1.encoder.layer.0.attention.output.dense.weight False
	 l1.encoder.layer.0.attention.output.dense.bias False
	 l1.encoder.layer.0.attention.output.LayerNorm.weight False
	 l1.encoder.layer.0.attention.output.LayerNorm.bias False
	 l1.encoder.layer.0.intermediate.dense.weight False
	 l1.encoder.layer.0.intermediate.dense.bias False
	 l1.encoder.layer.0.output.dense.weight False
	 l1.encoder.layer.0.output.dense.bias False
	 l1.encoder.layer.0.output.LayerNor

In [30]:
pos_weights = torch.tensor(pos_weights).to(device)


def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss(pos_weight=pos_weights)(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [33]:
checkpoint_path = cwd+'/checkpoints/PRE_FSJ_lstm_lr0.00005_emb_unfreeze.pt'


wandb.init(project='nlp-project', name='PRE_FSJ_lstm_lr0.00005_emb_unfreeze')
trained_model = train_model(1, 30, np.Inf, train_loader, valid_loader, model, \
                      optimizer, checkpoint_path) 
wandb.finish()

epoch,▁
epoch_train_loss,▁
epoch_val_loss,▁
global_step,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
train_loss,▄▄▂▃▆█▃█▁▆▇▃▅▄▄▂▅▆
epoch,1
epoch_train_loss,0.95606
epoch_val_loss,0.9556
global_step,18
train_loss,0.96036


############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.956004 	Average Validation Loss: 0.955542
Validation loss decreased (inf --> 0.955542).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.955977 	Average Validation Loss: 0.955482
Validation loss decreased (0.955542 --> 0.955482).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch_train_loss,███▇▇▇▆▆▆▆▄▅▅▃▅▄▃▃▃▃▂▂▂▃▂▂▂▂▁▂
epoch_val_loss,██▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
global_step,▄▇▆▅▁█▇▃▂▁▅▄▃▂▅▅▄▇▆▅▁█▇▃▂▁█▄▃▂▅▅▄▇▆▅▁█▇▆
train_loss,▄▂▄▇▅▇▅▆█▆▂▃▂▅▅▇▅▄▂▄▅▆██▆▁▆▇▅▂▆▆▄▄▆▄▁▄▃▆
epoch,30
epoch_train_loss,0.95522
epoch_val_loss,0.95444
global_step,18
train_loss,0.96228


### EVALUATION

In [ ]:
# model with LSTM layer
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.lstm = nn.LSTM(input_size=768,
                            hidden_size=128,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)
        self.l3 = torch.nn.Linear(256, 6)
    
    def forward(self, ids, mask, token_type_ids):
        _, output= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output = self.l2(output).unsqueeze(1)
        output, _ = self.lstm(output)
        output = self.l3(output).squeeze(1)
        return output
        
## model without LSTM layer
# class BERTClass(torch.nn.Module):
#     def __init__(self):
#         super(BERTClass, self).__init__()
#         self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
#         self.l2 = torch.nn.Dropout(0.3)
#         self.l3 = torch.nn.Linear(768, 6)
    
#     def forward(self, ids, mask, token_type_ids):
#         _, output= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
#         output = self.l2(output)
#         output = self.l3(output)
#         return output

model = BERTClass()

In [110]:
# checkpoint_path = '/home/svetlanamaslenkova/Documents/nlp_project/checkpoints/pretrained_model_full_ekman_lstm.pt'

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss(pos_weight=pos_weights)(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


In [ ]:
load_ckp(checkpoint_path, model=model, optimizer=optimizer, gpu=False)

In [100]:
device = 'cpu'
model = model.to(device)

In [103]:
sigmoid = torch.nn.Sigmoid()
targets_list = []
preds_list = []

for batch_idx, data in enumerate(test_loader):
    #print('yyy epoch', batch_idx)
    # ids = data['ids'].to(device, dtype = torch.long)
    # mask = data['mask'].to(device, dtype = torch.long)
    # token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    # targets = data['targets'].to(device, dtype = torch.float)
    ids = data['ids'].to(device)
    mask = data['mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    targets = data['targets'].to(device)

# outputs  = model_ckp(ids, mask, token_type_ids)

    outputs = sigmoid(model(ids, mask, token_type_ids))
    val_preds = (np.array(outputs.cpu().detach().numpy()) > 0.5).astype(int)
    targets = np.array(targets)
    # print(targets.shape)
    # print(val_preds.shape)

    if batch_idx == 0:
        targets_list = targets
        preds_list = val_preds
    else:
        targets_list = np.concatenate((targets_list, targets))
        preds_list = np.concatenate((preds_list, val_preds))

In [104]:
from sklearn import metrics

accuracy = np.round(metrics.accuracy_score(targets_list, preds_list), 2)
f1_score_micro = np.round(metrics.f1_score(targets_list, preds_list, average='micro'), 2)
f1_score_macro = np.round(metrics.f1_score(targets_list, preds_list, average='macro'), 2)
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.8
F1 Score (Micro) = 0.82
F1 Score (Macro) = 0.79


In [105]:
print(classification_report(targets_list, preds_list, output_dict=False))

              precision    recall  f1-score   support

           0       0.68      0.73      0.70       146
           1       0.85      0.76      0.80       342
           2       0.85      0.88      0.87       410

   micro avg       0.82      0.81      0.82       898
   macro avg       0.79      0.79      0.79       898
weighted avg       0.82      0.81      0.82       898
 samples avg       0.82      0.81      0.82       898



/home/svetlana.maslenkova/.conda/envs/nlp_project_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [106]:
cf = metrics.multilabel_confusion_matrix(targets_list, preds_list)


array([[[688,  50],
        [ 40, 106]],

       [[496,  46],
        [ 81, 261]],

       [[412,  62],
        [ 50, 360]]])

In [107]:
y_true = {}
y_pred = {}
if ekman_emotions:
    y_true['joy'], y_pred['joy'] = targets_list[:, 0], preds_list[:,0]
    y_true['fear'], y_pred['fear']= targets_list[:, 1], preds_list[:,1]
    y_true['anger'], y_pred['anger'] = targets_list[:, 2], preds_list[:,2]
    y_true['surprise'], y_pred['surprise'] = targets_list[:, 3], preds_list[:,3]
    y_true['disgust'], y_pred['disgust'] = targets_list[:, 4], preds_list[:,4]
    y_true['sadness'], y_pred['sadness'] = targets_list[:, 5], preds_list[:,5]
if sentiment:
    y_true['positive'], y_pred['positive'] = targets_list[:, 0], preds_list[:,0]
    y_true['negative'], y_pred['negative']= targets_list[:, 1], preds_list[:,1]
    y_true['neutral'], y_pred['neutral'] = targets_list[:, 2], preds_list[:,2]
    y_true['mixed'], y_pred['mixed'] = targets_list[:, 3], preds_list[:,3]

if full_ekman_emotions:
    y_true['anger'], y_pred['anger'] = targets_list[:, 0], preds_list[:,0]
    y_true['disguist'], y_pred['disguist']= targets_list[:, 1], preds_list[:,1]
    y_true['joy'], y_pred['joy'] = targets_list[:, 2], preds_list[:,2]
    y_true['sadness'], y_pred['sadness'] = targets_list[:, 3], preds_list[:,3]
    y_true['surprise'], y_pred['surprise'] = targets_list[:, 4], preds_list[:,4]
    y_true['neutral'], y_pred['neutral'] = targets_list[:, 5], preds_list[:,5]

if sentiment_3:
    y_true['positive'], y_pred['positive'] = targets_list[:, 0], preds_list[:,0]
    y_true['negative'], y_pred['negative']= targets_list[:, 1], preds_list[:,1]
    y_true['ambiguous'], y_pred['ambiguous'] = targets_list[:, 2], preds_list[:,2]

if FSJ_emotions:
    y_true['fear'], y_pred['fear'] = targets_list[:, 0], preds_list[:,0]
    y_true['sadness'], y_pred['sadness']= targets_list[:, 1], preds_list[:,1]
    y_true['joy'], y_pred['joy'] = targets_list[:, 2], preds_list[:,2]

In [108]:
f1 = {}
if ekman_emotions:
    emotions = ['joy',	'fear',	'anger',	'surprise',	'disgust',	'sadness']
if sentiment:
    emotions = ['positive',	'negative',	'neutral',	'mixed']
if full_ekman_emotions:
    emotions = ['anger',	'disguist',	'joy',	'sadness',	'surprise',	'neutral']
if sentiment_3:
    emotions = ['positive',	'negative',	'ambiguous']
if FSJ_emotions:
    emotions = ['fear', 'sadness', 'joy']


for emotion in emotions:
    f1[emotion] = np.round(metrics.f1_score(y_true[emotion], y_pred[emotion]), 2)
    print(f'F1-score for {emotion}: {f1[emotion]}')

F1-score for fear: 0.7
F1-score for sadness: 0.8
F1-score for joy: 0.87
